In [31]:
from funcoesTermosol import *

In [32]:
dados = importa('entrada.xls')
dados

(3,
 array([[0. , 0. , 0.3],
        [0. , 0.4, 0.4]]),
 3,
 array([[1.0e+00, 2.0e+00, 2.1e+11, 2.0e-04],
        [2.0e+00, 3.0e+00, 2.1e+11, 2.0e-04],
        [3.0e+00, 1.0e+00, 2.1e+11, 2.0e-04]]),
 2,
 array([[   0.],
        [   0.],
        [   0.],
        [   0.],
        [ 150.],
        [-100.]]),
 3,
 array([[0.],
        [2.],
        [3.]]))